## Coopetition - Muon id classification 

S. Ek-In, C. Praz


[x] Select features and find new features \
[ ] Add Scaling to wide range variables \
[ ] CatBoost \
[x] Scale weight - NOT USED

-> Need the module swifter with fsspec==0.3.3 , if the version is newer than this, the code might break



### Import part 


In [1]:
import os
import pandas as pd
import xgboost
import utils
import scoring
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

make_scorer(get_rejection_at_efficiency, needs_threshold=True, threshold=0.9)


### Download dataset

In [2]:
# The datasets are available in CoCalc in ~/share/data/I-coopetition-muon-id/
# Test
# ! wget --content-disposition https://codalab.coresearch.club/my/datasets/download/dd6255a1-a14b-4276-9a2b-db7f360e01c7
# Train
# ! wget --content-disposition https://codalab.coresearch.club/my/datasets/download/3a5e940c-2382-4716-9ff7-8fbc269b98ac

### Step 1 Load Data

In [3]:
DATA_PATH = "~/share/data/I-coopetition-muon-id/"
columns = utils.SIMPLE_FEATURE_COLUMNS + ["id", "label", "weight", "sWeight", "kinWeight"]
train = utils.load_full_train_csv(DATA_PATH, 600000)
print(train.shape)

(600000, 76)


### Step 2 Data Preprocessing 

In [4]:
# Main Setting 

weight_name = "weight"
num_sample = 10
train_vis = train.head(num_sample)

In [5]:
'''
    Utils functions
'''

#classes = [0, 1]
#columns = ['P', 'PT','ncl[0]', 'avg_cs[0]', 'ndof', 'MatchedHit_TYPE[0]', 'MatchedHit_TYPE[1]', 'MatchedHit_TYPE[2]']


def visualize(feature, target, weights, num_bins=100):
    classes = np.unique(target)
    bins = np.linspace(feature.min(), feature.max(), num_bins + 1)
    
    # Plot all class 
    for c in classes:
        selection = (target == c)
        plt.hist(feature[selection], bins = bins, label = c, alpha = 0.5, weights = weights[selection])
    plt.legend()


In [6]:
print(train.columns)

Index(['ncl[0]', 'ncl[1]', 'ncl[2]', 'ncl[3]', 'avg_cs[0]', 'avg_cs[1]',
       'avg_cs[2]', 'avg_cs[3]', 'ndof', 'MatchedHit_TYPE[0]',
       'MatchedHit_TYPE[1]', 'MatchedHit_TYPE[2]', 'MatchedHit_TYPE[3]',
       'MatchedHit_X[0]', 'MatchedHit_X[1]', 'MatchedHit_X[2]',
       'MatchedHit_X[3]', 'MatchedHit_Y[0]', 'MatchedHit_Y[1]',
       'MatchedHit_Y[2]', 'MatchedHit_Y[3]', 'MatchedHit_Z[0]',
       'MatchedHit_Z[1]', 'MatchedHit_Z[2]', 'MatchedHit_Z[3]',
       'MatchedHit_DX[0]', 'MatchedHit_DX[1]', 'MatchedHit_DX[2]',
       'MatchedHit_DX[3]', 'MatchedHit_DY[0]', 'MatchedHit_DY[1]',
       'MatchedHit_DY[2]', 'MatchedHit_DY[3]', 'MatchedHit_DZ[0]',
       'MatchedHit_DZ[1]', 'MatchedHit_DZ[2]', 'MatchedHit_DZ[3]',
       'MatchedHit_T[0]', 'MatchedHit_T[1]', 'MatchedHit_T[2]',
       'MatchedHit_T[3]', 'MatchedHit_DT[0]', 'MatchedHit_DT[1]',
       'MatchedHit_DT[2]', 'MatchedHit_DT[3]', 'Lextra_X[0]', 'Lextra_X[1]',
       'Lextra_X[2]', 'Lextra_X[3]', 'Lextra_Y[0]', 'Lextra_

In [7]:
# Looking for high level parameters 
import swifter 
def Get_closest_hits(data):
    
    closest_hits_features = data.swifter.apply(utils.find_closest_hit_per_station, result_type="expand", axis=1)
    closest_hits_features.columns = ["closest_{}".format(ind) for ind in range(len(closest_hits_features.columns))]
    
    return closest_hits_features

close_hits = Get_closest_hits(train)
train_mod = pd.concat([train, close_hits], axis = 1)

# Save to files for backing up
close_hits.to_pickle('train_closest_hit.pkl')



In [27]:
# Scale product of sWeight
import pandas as pd
pd.options.mode.chained_assignment = None 

def scale_sweight(data, edge_weight = None):
    if edge_weight != None:
        data_mod = data[data['weight'].abs() < edge_weight]
        data_mod['scale_weight'] = (data['weight'] + edge_weight)/(2 * edge_weight)
        data_mod['scale_weight'] = data_mod['scale_weight'] * (data_mod['weight'].sum() / data_mod['scale_weight'].sum())
    
        return data_mod
    else:
        data_mod['scale_weight'] = (data_mod['weight'] + data_mod['weight'].min()) (data_mod['weight'].sum() / data_mod['scale_weight'].sum())

# Scale sweight
#train_mod = scale_sweight(train_mod, edge_weight = None)
    
# Add new features 
def feat_PZ(data): 
    return np.sqrt(data['P'] ** 2  - data['PT'] ** 2)



train_mod['PZ'] = feat_PZ(train_mod)

In [28]:
train_mod['P']/train_mod['PZ']

id
0         1.026119
1         1.012146
2         1.020520
3         1.003271
4         1.003093
            ...   
599995    1.045375
599996    1.010911
599997    1.002369
599998    1.002907
599999    1.023048
Length: 600000, dtype: float32

In [29]:
# Visualisation 

for column in []:
    plt.figure()
    visualize(train[column][:num_sample], train['label'][:num_sample], train[weight_name][:num_sample])
    plt.title(column)


## Note: 
 - Acc = 0.70, train_cols with close hit, add PZ, Model: XGBoost LR = 0.1 


In [30]:
# Def used columns 
train_cols = ['ncl[0]', 'ncl[1]', 'ncl[2]', 'ncl[3]', 'avg_cs[0]', 'avg_cs[1]',
       'avg_cs[2]', 'avg_cs[3]', 'ndof', 'MatchedHit_TYPE[0]',
       'MatchedHit_TYPE[1]', 'MatchedHit_TYPE[2]', 'MatchedHit_TYPE[3]',
       'MatchedHit_X[0]', 'MatchedHit_X[1]', 'MatchedHit_X[2]',
       'MatchedHit_X[3]', 'MatchedHit_Y[0]', 'MatchedHit_Y[1]',
       'MatchedHit_Y[2]', 'MatchedHit_Y[3]', 'MatchedHit_Z[0]',
       'MatchedHit_Z[1]', 'MatchedHit_Z[2]', 'MatchedHit_Z[3]',
       'MatchedHit_DX[0]', 'MatchedHit_DX[1]', 'MatchedHit_DX[2]',
       'MatchedHit_DX[3]', 'MatchedHit_DY[0]', 'MatchedHit_DY[1]',
       'MatchedHit_DY[2]', 'MatchedHit_DY[3]', 'MatchedHit_DZ[0]',
       'MatchedHit_DZ[1]', 'MatchedHit_DZ[2]', 'MatchedHit_DZ[3]',
       'MatchedHit_T[0]', 'MatchedHit_T[1]', 'MatchedHit_T[2]',
       'MatchedHit_T[3]', 'MatchedHit_DT[0]', 'MatchedHit_DT[1]',
       'MatchedHit_DT[2]', 'MatchedHit_DT[3]', 'NShared', 'Mextra_DX2[0]',
       'Mextra_DX2[1]', 'Mextra_DX2[2]', 'Mextra_DX2[3]', 'Mextra_DY2[0]',
       'Mextra_DY2[1]', 'Mextra_DY2[2]', 'Mextra_DY2[3]', 'FOI_hits_N',
       'PT', 'PZ'] + close_hits.columns.tolist()
target_col = ['label', 'weight']

In [14]:
import re

def prepare_data(data):
    regex = re.compile(r"\[|\]|<", re.IGNORECASE)
    data.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in data.columns.values]



In [15]:
# Check Heading data
train.head()
X_mod, y_mod = train_mod[train_cols], train_mod[target_col]

# Rename - ignore [] 
prepare_data(X_mod)

# Splitting 
X_train, X_val, y_train, y_val = train_test_split(X_mod, y_mod, test_size=0.25, shuffle=True, random_state=2342234)

# Scale 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
'''
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
pca = PCA()
X_train = pca.fit_transform(X_train)
X_val = pca.transform(X_val)
'''



'\nscaler = StandardScaler()\nX_train = scaler.fit_transform(X_train)\nX_val = scaler.transform(X_val)\npca = PCA()\nX_train = pca.fit_transform(X_train)\nX_val = pca.transform(X_val)\n'

In [16]:
print(X_train)

        ncl_0_  ncl_1_  ncl_2_  ncl_3_  avg_cs_0_  avg_cs_1_  avg_cs_2_  \
id                                                                        
71278     44.0     6.0    11.0     6.0   2.704545   1.666667   1.363636   
357995   102.0    16.0     4.0    13.0   3.303921   3.812500   1.000000   
547395    25.0     8.0    18.0    14.0   1.520000   1.375000   1.944444   
533566    87.0    12.0     9.0     6.0   2.620690   1.500000   1.000000   
17230    102.0    39.0    22.0    11.0   2.205882   2.384615   1.272727   
...        ...     ...     ...     ...        ...        ...        ...   
373825    33.0     4.0     7.0    11.0   4.303030   2.500000   1.000000   
275161    79.0    39.0    14.0    13.0   2.050633   2.461539   1.285714   
171712    66.0    15.0     7.0    11.0   1.818182   1.200000   1.142857   
533939    15.0    11.0     5.0    11.0   2.200000   2.363636   1.400000   
388600    82.0    18.0     7.0    12.0   2.134146   2.277778   1.285714   

        avg_cs_3_  ndof 

### Step 3 Training part 

In [17]:
# Model Arch
n_trees = 100
model = xgboost.XGBClassifier(n_estimators = n_trees, nthread=-1, learning_rate = 0.1)


In [18]:
from sklearn.metrics import accuracy_score
import joblib
# Trainning part

score_best = 0
model_best = None
lr_best    = None

lrs = [0.1]

for lr in lrs:
    print("trainning with lr = {}".format(lr))
    model = xgboost.XGBClassifier(n_estimators = n_trees, nthread=-1, learning_rate = lr)

    model.fit(X_train.iloc[:, :],
              y_train['label'].values,
              #sample_weight=y_train.scale_weight.values,
              #verbose=True,
             )
    ''' train with scaler and PCA 
    
    model.fit(X_train,
              y_train['label'].values,
              sample_weight=y_train.scale_weight.values,
              verbose=True,
             )
    '''
    validation_predictions = model.predict_proba(X_val)[:, 1]
    #model_score = scoring.rejection90(y_val.label.values, validation_predictions, sample_weight = y_val.scale_weight.values)
    model_score = scoring.rejection90(y_val.label.values, validation_predictions)


    print("NN: {} , Test accuracy: {}".format(lr, model_score))
    if model_score > score_best :
        model_best = model
        score_best = model_score
        lr_best    = lr

# Save Model 
joblib.dump(model_best, 'Model_Best.pkl')


trainning with lr = 0.1


NN: 0.1 , Test accuracy: 0.7057152573566184


['Model_Best.pkl']

In [19]:
print(score_best)

0.7057152573566184


In [23]:
validation_predictions = model.predict_proba(X_val)[:, 1]
model_score = scoring.rejection90(y_val.label.values, validation_predictions)

In [0]:
print(model_score)

In [21]:
# Further test on 20% of original training 
Eval_train = train.head(int(0.5 * train.shape[0]))

# Prep sample 
close_hits = Get_closest_hits(Eval_train)
Eval_train = pd.concat([Eval_train, close_hits], axis = 1)
Eval_train['PZ'] = feat_PZ(Eval_train)

Eval_X = Eval_train[train_cols]
prepare_data(Eval_X)


# Predict and save file 
Eval_true =  Eval_train[['label']]
predictions = model_best.predict_proba(Eval_X)[:, 1]
model_score = scoring.rejection90(Eval_true.label.values, predictions, Eval_train.weight.values)

In [22]:
print(model_score)

0.730293198462288


### Step 4 Predict on the whole test set and prepare submission



In [0]:

# Read data
DATA_PATH = "~/share/data/I-coopetition-muon-id/"
test = utils.load_full_test_csv(DATA_PATH, None)


# Transform data 
close_hits_test = Get_closest_hits(test)
test_mod = pd.concat([test, close_hits_test], axis = 1)

test_mod['PZ'] = feat_PZ(test_mod)
X_test = test_mod[train_cols]
prepare_data(X_test)


# Predict and save file 
predictions = model_best.predict_proba(X_test)[:, 1]

compression_opts = dict(method='zip',
                        archive_name='submission.csv')  
pd.DataFrame(data={"prediction": predictions}, index=test.index).to_csv(
    "submission.zip", index_label=utils.ID_COLUMN, compression=compression_opts)